In [39]:
import firebase_admin
from firebase_admin import credentials, firestore
import numpy as np
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import VotingClassifier
import joblib
# Initialize Firebase
if not firebase_admin._apps:
    cred = credentials.Certificate(r"C:\Users\PRAJWAL CHANDEER\Dropbox\PC\Desktop\SpeechRecog\speakerrecog.json")  # Replace with your file path
    firebase_admin.initialize_app(cred)

# Connect to Firestore
db = firestore.client()

# Fetch data from Firestore
features_list = []
usernames = []

# Retrieve documents from Firestore collection
docs = db.collection('audioFeatures').stream()
for doc in docs:
    data = doc.to_dict()

    # Parse features
    mfcc = list(map(float, data['mfcc'].split(",")))  # Convert comma-separated string to list of floats
    chroma = list(map(float, data['chroma'].split(",")))
    energy = float(data['energy'])
    spectral_features = [
        data['spectralCentroid'],
        data['spectralFlatness'],
        data['spectralKurtosis'],
        data['spectralRolloff'],
        data['spectralSkewness'],
        data['spectralSlope'],
        data['spectralSpread'],
        data['zeroCrossingRate'],
    ]

    # Combine all features into a single list
    combined_features = mfcc + chroma + spectral_features + [energy]
    features_list.append(combined_features)
    usernames.append(data['username'])

# Convert to NumPy arrays
X = np.array(features_list)  # Features
y = np.array(usernames)      # Labels
print(X.shape)
print(y.shape)
# Encode usernames into numerical labels
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

(64, 34)
(64,)


In [45]:


# Define models and parameters
svm_params = {
    'C': [0.1, 1, 10],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['rbf', 'linear']
}

rf_params = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20],
    'class_weight': ['balanced'],
    'random_state': [12]
}

knn_params = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Perform GridSearchCV/RandomizedSearchCV
kf = KFold(n_splits=5, shuffle=True, random_state=12)

svm_model = GridSearchCV(SVC(), svm_params, cv=kf, scoring='accuracy')
svm_model.fit(X_scaled, y_encoded)

rf_model = RandomizedSearchCV(RandomForestClassifier(), rf_params, cv=kf, scoring='accuracy', n_iter=10, random_state=26)
rf_model.fit(X_scaled, y_encoded)

knn_model = GridSearchCV(KNeighborsClassifier(), knn_params, cv=kf, scoring='accuracy')
knn_model.fit(X_scaled, y_encoded)

# Get best models
best_svm = svm_model.best_estimator_
best_rf = rf_model.best_estimator_
best_knn = knn_model.best_estimator_

# Voting Classifier
voting_model = VotingClassifier(estimators=[
    ('svm', best_svm),
    ('rf', best_rf),
    ('knn', best_knn)
], voting='hard')

# Evaluate with Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=120)
voting_model.fit(X_train, y_train)

# Predict and Evaluate
y_pred = voting_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(X.shape)
print(y_test)
print(y_pred)
print(f"Voting Classifier Accuracy: {accuracy * 100:.2f}%")

# Save models and encoders
joblib.dump(label_encoder, r"C:\Users\PRAJWAL CHANDEER\Dropbox\PC\Desktop\SpeechRecog\frontend\src\components\label_encoder.pkl")
joblib.dump(voting_model, r"C:\Users\PRAJWAL CHANDEER\Dropbox\PC\Desktop\SpeechRecog\frontend\src\components\voting_model.pkl")
joblib.dump(scaler, r"C:\Users\PRAJWAL CHANDEER\Dropbox\PC\Desktop\SpeechRecog\frontend\src\components\scaler.pkl")


C:\Python311\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


(64, 34)
[0 2 0 2 0 1 1 0 2 0 2 2 1]
[0 2 0 2 0 1 1 0 2 0 2 2 1]
Voting Classifier Accuracy: 100.00%


['C:\\Users\\PRAJWAL CHANDEER\\Dropbox\\PC\\Desktop\\SpeechRecog\\frontend\\src\\components\\scaler.pkl']